In [50]:
# Dependencies
import requests
from config import api_key, CLIENT_ID, CLIENT_SECRET
import json
import pprint as pp
import pandas as pd
import time
import lyricsgenius

In [38]:
# Request Access Token for Spotify API calls.
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [39]:
# Build query URL
# Access the playlist here: https://open.spotify.com/playlist/1Jczb1SckWxYiqrIzv0jsl?si=560c28541ccb42f5

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

url = "https://api.spotify.com/v1/"

play_id = "1Jczb1SckWxYiqrIzv0jsl?si=560c28541ccb42f5"

query_url = url+'playlists/'+play_id+'tracks'

In [46]:
# Get playlist data and initialize arrays for DataFrame assembly.
json_req = requests.get(query_url, headers=headers)
response = json_req.json()

track_ids = []
track_names = []
artist_names = []
acousticness = []
danceability = []
energy = []
instrumentalness = []
liveness = []
loudness = []
speechiness = []
tempo = []
valence = []

#Assign each genre as listed in the playlist.
genres = []
for j in range (1,9):
    for i in range(1,16):
        genres.append(j)

In [47]:
#Append track ID and related song data, then make separate API call to find the track's audio features.
first_part = True
while True:
    if first_part == True:
        for item in response['tracks']['items']:
            track_names.append(item['track']['name'])
            track_ids.append(item['track']['id']) 
            artist_names.append(item['track']['album']['artists'][0]['name'])
            #Make new API call on track's audio features based on each track's ID.
            audio_query_url = url+'audio-features/'+item['track']['id']
            json_req = requests.get(audio_query_url, headers=headers)
            audio_response = json_req.json()
            acousticness.append(audio_response['acousticness'])
            danceability.append(audio_response['danceability'])
            energy.append(audio_response['energy'])
            instrumentalness.append(audio_response['instrumentalness'])
            liveness.append(audio_response['liveness'])
            loudness.append(audio_response['loudness'])
            speechiness.append(audio_response['speechiness'])
            tempo.append(audio_response['tempo'])
            valence.append(audio_response['valence'])
        #Switch to second half of API request (Index max of 100 tracks in playlist per request)
        audio_response = response['tracks']['next']
        first_part = False
        time.sleep(3)
    else:
        req = requests.get(audio_response, headers=headers)
        data = req.json()
        for item in data['items']:
            track_names.append(item['track']['name'])
            track_ids.append(item['track']['id']) 
            artist_names.append(item['track']['album']['artists'][0]['name'])
            #Make new API call on track's audio features based on each track's ID.
            audio_query_url = url+'audio-features/'+item['track']['id']
            json_req = requests.get(audio_query_url, headers=headers)
            audio_response = json_req.json()
            acousticness.append(audio_response['acousticness'])
            danceability.append(audio_response['danceability'])
            energy.append(audio_response['energy'])
            instrumentalness.append(audio_response['instrumentalness'])
            liveness.append(audio_response['liveness'])
            loudness.append(audio_response['loudness'])
            speechiness.append(audio_response['speechiness'])
            tempo.append(audio_response['tempo'])
            valence.append(audio_response['valence'])
        break
#Assemble the DataFrame.
df = pd.DataFrame({ 'Track Name':track_names, 'Artist':artist_names, 'Track ID':track_ids, 'Genres':genres, 'Acousticness':acousticness,
                  'Danceability':danceability, 'Energy':energy, 'Instrumentalness':instrumentalness, 'Liveness':liveness,'Loudness':loudness,
                  'Speechiness':speechiness, 'Tempo':tempo, 'Valence':valence})



In [48]:
df.head(20)

,Track Name,Artist,Track ID,Genres,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence
0,The Real Slim Shady,Eminem,3yfqSUWxFvZELEM4PmlwIR,1,0.030200,0.949,0.661,0.000000,0.0454,-4.244,0.0572,104.504,0.760
1,MotorSport,Migos,4wFjTWCunQFKtukqrNijEt,1,0.030500,0.904,0.518,0.000000,0.3250,-5.320,0.1830,137.996,0.188
2,In Da Club,50 Cent,5Mcvgt2GbxxCp6cz3Kfjey,1,0.325000,0.906,0.663,0.000000,0.0883,-3.955,0.3550,90.042,0.847
3,Gangsta's Paradise,Coolio,1DIXPcTDzTj8ZMHt3PDt8p,1,0.065500,0.647,0.514,0.000000,0.3980,-10.050,0.0593,79.975,0.386
4,rockstar,Post Malone,19bX4zmC2zDben2ldhoRB3,1,0.145000,0.594,0.507,0.000871,0.1320,-6.141,0.0816,159.729,0.137
5,HUMBLE.,Kendrick Lamar,131OLY5J8XyfGuSjXRiTRM,1,0.000253,0.911,0.566,0.000075,0.0956,-6.703,0.1350,150.039,0.374
6,Black and Yellow,Wiz Khalifa,3VP78k3jzm0Q5OM08E383k,1,0.061900,0.685,0.832,0.000000,0.2730,-4.548,0.0595,164.070,0.574
7,WAP (feat. Megan Thee Stallion),Cardi B,051wt8AyLFgYnVuberd3vO,1,0.027200,0.934,0.443,0.000000,0.0889,-7.541,0.4100,133.026,0.359
8,Panini,Lil Nas X,6fTt0CH2t0mdeB2N9XFG5r,1,0.342000,0.703,0.594,0.000000,0.1230,-6.146,0.0752,153.848,0.475
9,Reflections,Bliss n Eso,5Wf0EKbK4AB6xtSbsqL4r2,1,0.725000,0.497,0.605,0.000000,0.2380,-8.625,0.0468,77.811,0.824


In [51]:
#Append lyric information from Genius API.
#Import ID and Secret.
CLIENT_ID = 'mVxsgubdxvPwivQOYFWt5FCskAPVyDRl7ZQnt4PPvlBaPHfbTATSqoqt2KKVjji1'
CLIENT_SECRET = 'u6bH4rznsJK0A80jHqIkJ3I6XxJr6IbtY0EX7T4aDDre5wCREj9ywuA5khlbyNM9AeKLoSFcYjYLA6vCTOKhPg'
genius = lyricsgenius.Genius('3Ee9v1MsjFxlAwwA0OkZWNJEHWRBhYWqpp5TwwWk_UX70_P36TH7StPKeBLt88EI')

In [ ]:
#For loop that iterates through first column of df, searches for 
lyrics = []
failed = []
i = 0
for index, row in df.iterrows():
    song_title = row['Track Name']
    try:
        songs = genius.search_song(song_title)
        lyrics.append(songs.lyrics)
        time.sleep(2)
    except:
        lyrics.append(0)
        print('Error retrieving lyrics. Retrying on next track...')
        failed.append(song_title)
        time.sleep(5)
        pass
pp.pprint(lyrics)

In [ ]:
#Append lyric information to df.
df['Lyrics'] = lyrics
df.head()